In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

### A placeholder GPT model architecture class

In [ ]:
import torch

class DummyGPTModel(torch.nn.Module):
    def __init__(self, cfg):
        self.tok_emb = torch.nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = torch.nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = torch.nn.Dropout(cfg["drop_rate"])
        self.transformer_blocks = torch.nn.Sequential([
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        ])
        self.final_norm = torch.nn.DummyLayerNorm(cfg["emb_dim"])
        self.out_head = torch.nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(torch.nn.Module):
    def __init(self, cfg):
        super.__init__()

    def forward(self, x):
        return x
    
class DummyLayerNorm(torch.nn.Module):
    def __init__(self, normalized_shape, eps=1e-5): 
        super().__init__()

    def forward(self, x):
        return x